Emma Klemets, Nov 2022   
To investigate if we have aligned the old and new data properly.   
A work in progress

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pandas as pd
import sys, os
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.ticker import FormatStrFormatter

sys.path.insert(1, '../PrettyPlotTools/')
import ParticlePlottingFunctions as ppf
import seaborn as sns

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
colNames = ['x', 'y', 'z','B_x', 'B_y', 'B_z']

file_22 = "data_export/map_referencedMSR_fall2022_together_interp50.txt"
data_22 = pd.read_csv(file_22, comment='#', sep='\t', names=colNames)

file_19 = "../MagMapping-export_TH_2019/data_export/map_referencedMSR_interp50.txt"
data_19 = pd.read_csv(file_19, comment='#', sep='\t', names=colNames)

In [62]:
cut_22 = "data_export/map_referencedMSR_fall2022_together_CUT_interp50.txt"
dataCut_22 = pd.read_csv(cut_22, comment='#', sep='\t', names=colNames)

#the data in this file for the B field is 
cut_19 = "../MagMapping-export_TH_2019/data_export/map_referencedMSR_interp_CUT_interp50.txt"
dataCut_19 = pd.read_csv(cut_19, comment='#', sep='\t', names=colNames)

In [63]:
mins_19 = np.min(dataCut_19)
max_19 = np.max(dataCut_19)

mins_22 = np.min(dataCut_22)
max_22 = np.max(dataCut_22)

# print("2019 limits:")
# print(mins_19,'\n', max_19)
# print("2022 limits:")
# print(mins_22, '\n',max_22)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [80]:
def Shiftdata_compare(data_19, data_22, shift, comp='x'): 
    #this does not work as no points line up after any shifts
    data_22_Shift = data_22.copy()
    
    data_22_Shift[comp] = data_22_Shift[comp] + shift
    print(data_22_Shift)
    
    df_compared = data_19.join(data_22_Shift, rsuffix='_22', how='outer')

    df_compared = df_compared.where((df_compared['x']==df_compared['x_22']) & 
                                          (df_compared['y']==df_compared['y_22']) &
                                          (df_compared['z']==df_compared['z_22']))
    # print(df_compared)
    
    df_compared = df_compared.drop(columns=['x_22', 'y_22', 'z_22'])

    df_compared['dB_x'] = df_compared['B_x'] - df_compared['B_x_22'] 
    df_compared['dB_y'] = df_compared['B_y'] - df_compared['B_y_22'] 
    df_compared['dB_z'] = df_compared['B_z'] - df_compared['B_z_22'] 
    
    
    return df_compared

In [81]:
#I think I need to shift the x/y/z parts by exactly the right amount to get some points to
#match up, or else I need to go back to my interpolation parts
print(dataCut_22['x'].diff(1).unique())

df_compare_2 = Shiftdata_compare(dataCut_19, dataCut_22, shift=-4.36141612, comp='x')
# print(df_compare_2)
print(df_compare_2[~df_compare_2['x'].isnull()])

# df_compare_2.equals(df_compare)

[          nan    0.            4.36141612    4.36141612    4.36141612
    4.36141612    4.36141612    4.36141612    4.36141612    4.36141612
    4.36141612    4.36141612    4.36141612    4.36141612    4.36141612
 -213.70939   ]
                 x           y          z        B_x        B_y         B_z
0       -94.461416 -174.711330 -150.75165 -94.285496 -94.418055 -209.192935
1       -94.461416 -173.076666 -150.75165 -93.885800 -92.701266 -208.856689
2       -94.461416 -171.442002 -150.75165 -93.495098 -90.981172 -208.522847
3       -94.461416 -169.807338 -150.75165 -93.113186 -89.259196 -208.192510
4       -94.461416 -168.172674 -150.75165 -92.739623 -87.536901 -207.866626
...            ...         ...        ...        ...        ...         ...
124995  119.247974 -101.151446    9.25780  -3.481038 -40.968494 -198.856416
124996  119.247974  -99.516782    9.25780  -3.586228 -40.884611 -198.654345
124997  119.247974  -97.882118    9.25780  -3.687138 -40.794100 -198.442927
124998  119

In [64]:
# list(dataCut_19.columns)

df_compare = dataCut_19.join(dataCut_22, rsuffix='_22', how='outer')
#                               on=list(dataCut_22.columns))

# df_compare.loc[0]['x_19'] = 0
# display(df_compare.columns)
df_compare = df_compare.where((df_compare['x']==df_compare['x_22']) & 
                                      (df_compare['y']==df_compare['y_22']) &
                                      (df_compare['z']==df_compare['z_22']))
df_compare = df_compare.drop(columns=['x_22', 'y_22', 'z_22'])
# display(df_compare)

df_compare['dB_x'] = df_compare['B_x'] - df_compare['B_x_22'] 
df_compare['dB_y'] = df_compare['B_y'] - df_compare['B_y_22'] 
df_compare['dB_z'] = df_compare['B_z'] - df_compare['B_z_22'] 

# display(df_compare)

In [25]:
# titleS = "Difference of mapping data components \n 2019 data - 2022 data"
titleS = "What I think the correct placement is"


ppf.PlotComponents(df_compare_2, Compare=True, lims=None, title=titleS, Sample=None)
# plt.savefig('plots/compare_3C.png', bbox_inches='tight')
plt.show()

ValueError: vmin, vcenter, vmax must increase monotonically

AttributeError: 'Path3DCollection' object has no attribute '_offsets3d'

<Figure size 2000x600 with 3 Axes>

In [11]:
print("mean (x,y,z): ", np.mean(df_compare['dB_x']), np.mean(df_compare['dB_y']), 
      np.mean(df_compare['dB_z']))
print("STD (x,y,z): ", np.std(df_compare['dB_x']), np.std(df_compare['dB_y']), 
      np.std(df_compare['dB_z']))

mean (x,y,z):  -16.79683725262247 -6.654554952087327 5.52693342547019
STD (x,y,z):  8.56961242513111 7.211486929980489 13.09056371785573
